In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
# import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch import Tensor
from torch.nn.parameter import Parameter

import math
import warnings

In [3]:
def quantizeWeight(W,BW):
    W = torch.min(W,(1.0-(2**(-(BW-1.0))))*torch.ones_like(W))
    Wbs = []
    Wbi = torch.lt(W,torch.zeros_like(W)).float()
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = torch.ge(W,0.5*torch.ones_like(W)).float()
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = torch.ge(W,0.5*torch.ones_like(W)).float()
    for i in range(BW):#-1):
        j = BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = torch.gt(Wbs[j],1.5*torch.ones_like(Wbs[j])).float()
        Wbs[j] = Wbs[j]*torch.ne(Wbs[j],2.0*torch.ones_like(Wbs[j]))
    return Wbs

def reconstructWeight(Wbs,BW):
    W = torch.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = (0.5)**j
        if (j == 0):
            multiplier = -1.0
        W += Wbs[j] * multiplier
    return W

def quantize_activations(input):
    input = torch.min(torch.round(input*(2**6))*(2**(-6)) ,(1.0-(2**(-6)))*torch.ones_like(input))
    return input

In [27]:
weight = torch.rand((6,3,3,3))
inputs = torch.rand((1,3,12,12))

weight_q = reconstructWeight(quantizeWeight(weight,6),6)
inputs_q = quantize_activations(inputs)

# print(weight_q.size())
# print(weight_q[3:6,:,:,:].size())
# print(weight_q)
# print("a")
# print(weight_q[3:6,:,:,:])
# print("b")
# print(weight_q[0:3,:,:,:])

In [42]:
layer = nn.Conv2d(3,6,3)
layer_1 = nn.Conv2d(3,3,3)
layer_2 = nn.Conv2d(3,3,3)
layer.weight.data = weight_q
layer_1.weight.data = weight_q[0:3,:,:,:]

layer_2.weight.data = weight_q[3:6,:,:,:]


output = layer(inputs)
print(output[:,0,:,:])
print(output.size())
print('a')
output_p = torch.zeros((1,6,10,10))
output_p[:,0:3,:,:] = layer_1(inputs)
output_p[:,3:6,:,:] = layer_2(inputs)
print(output_p.size())
print(output_p[:,0,:,:])

tensor([[[5.8560, 6.7928, 6.0645, 5.2221, 6.2805, 6.5670, 8.5814, 8.2574,
          8.4040, 7.7292],
         [6.8762, 6.9639, 5.9899, 5.4987, 5.6990, 6.5812, 7.5143, 7.7410,
          8.4085, 7.7306],
         [6.2789, 7.1374, 6.1144, 6.0556, 5.3388, 5.8394, 7.5172, 8.0621,
          9.3200, 9.2684],
         [6.9430, 6.8681, 6.4720, 6.1372, 6.2095, 5.8142, 7.1482, 7.9811,
          8.3743, 7.9381],
         [6.4228, 7.7775, 7.9725, 7.4058, 6.1472, 6.4528, 7.0765, 7.8599,
          7.5082, 7.4778],
         [7.6175, 7.6876, 7.5986, 8.4094, 7.3208, 8.1489, 6.9872, 6.7055,
          6.1841, 7.0345],
         [7.7291, 7.2411, 8.3434, 8.6439, 6.4074, 6.2543, 6.3089, 7.1656,
          6.2531, 6.7290],
         [6.0199, 6.2960, 7.5069, 8.1723, 7.4958, 6.2618, 6.0522, 6.2374,
          6.7517, 8.3873],
         [6.0215, 5.9063, 6.5072, 8.6873, 6.4011, 5.8394, 6.2358, 6.2409,
          7.7586, 7.7819],
         [6.2081, 5.6946, 7.3523, 8.6155, 6.3912, 6.1928, 6.5183, 7.1748,
          8.0279,